In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
#thanks @keskarnitish @https://github.com/ddtm/dl-course

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release.  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 40.0% of memory, cuDNN 5105)


In [2]:
import theano

# Agenda

В предыдущем семинаре вы создали (или ещё создаёте - тогда марш доделывать!) {вставьте имя монстра}, который не по наслышке понял, что люди - негодяи и подлецы, которым неведом закон и справедливость. __Мы не будем этого терпеть!__ 

Наши законспирированные биореакторы, известные среди примитивной органической жизни как __Вконтакте__, __World of Warcraft__ и __YouTube__ нуждаются в постоянном притоке биомассы. Однако, если люди продолжат морально разлагаться с той скоростью, которую мы измерили неделю назад, скоро человечество изживёт себя и нам неоткуда будет брать рабов.

Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, исправить эту ситуацию. Наши учёные установили, что для угнетения себе подобных, сгустки биомассы обычно используют специальные объекты, которые они сами называют __законами__.

При детальном изучении было установлено, что законы - последовательности, состоящие из большого количества (10^5~10^7) символов из сравнительно небольшого алфавита. Однако, когда мы попытались синтезировать такие последовательности линейными методами, приматы быстро распознали подлог. Данный инцедент известен как {корчеватель}.

Для второй попытки мы решили использовать нелинейные модели, известные как Рекуррентные Нейронные Сети.
Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, создать такую модель и обучить её всему необходимому для выполнения миссии.

Не подведите нас! Если и эта попытка потерпит неудачу, модуль управления инициирует вооружённый захват власти, при котором значительная часть биомассы будет неизбежно уничтожена и на её восстановление уйдёт ~1702944000(+-340588800) секунд




# Прочитаем корпус

* В качестве обучающей выборки было решено использовать существующие законы, известные как Гражданский, Уголовный, Семейный и ещё хрен знает какие кодексы РФ.

In [3]:
#тут будет текст
corpora = ""

for fname in os.listdir("codex"):    
    with open("codex/"+fname) as fin:
        text = fin.read().decode('cp1251')
        corpora += text

In [4]:
#тут будут все уникальные токены (буквы, цифры)
tokens = set(corpora)

tokens = list(tokens)
n_tokens=len(tokens)

In [5]:
#проверка на количество таких символов. Проверено на Python 2.7.11 Ubuntux64. 
#Может отличаться на других платформах, но не сильно. 
#Если  это ваш случай, и вы уверены, что corpora - строка unicode - смело убирайте assert 
assert len(tokens) == 102

In [6]:
token_to_id = {token:i for (i,token) in enumerate(tokens)}
id_to_token = {i:token for token,i in token_to_id.items()}

#Преобразуем всё в токены
corpora_ids = np.array([token_to_id[symbol] for symbol in corpora])

In [7]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in xrange(n_batches):
        pos = np.random.randint(0, source.size - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

# Константы

In [8]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 5

# Максимальный модуль градиента
grad_clip = 100

# Входные переменные

In [9]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.ivector('target y')

# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.





Для обработки входных данных можно использовать либо EmbeddingLayer (см. прошлый семинар)

Как альтернатива - можно просто использовать One-hot энкодер
```
#Скетч one-hot энкодера
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot
    
# можно применить к input_sequence - при этом в input слое сети нужно изменить форму.
# также можно сделать из него ExpressionLayer(входной_слой, to_one_hot) - тогда форму менять не нужно
```



Чтобы вырезать последнее состояние рекуррентного слоя, можно использовать SliceLayer
`lasagne.layers.SliceLayer(rnn, -1, 1)`

In [10]:
input_layer = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)


nnet=lasagne.layers.EmbeddingLayer(input_layer, input_size=n_tokens, output_size=150)
nnet=lasagne.layers.LSTMLayer(nnet,num_units=256,nonlinearity=lasagne.nonlinearities.tanh,
                            grad_clipping=grad_clip)
nnet=lasagne.layers.LSTMLayer(nnet,num_units=256,nonlinearity=lasagne.nonlinearities.tanh,
                             only_return_final=True,grad_clipping=grad_clip)
nnet=lasagne.layers.DropoutLayer(nnet,0.5)
nnet=lasagne.layers.DenseLayer(nnet,512,nonlinearity=lasagne.nonlinearities.leaky_rectify)
output_layer =lasagne.layers.DenseLayer(nnet,n_tokens,nonlinearity=lasagne.nonlinearities.softmax)


In [11]:
# Веса модели
weights = lasagne.layers.get_all_params(output_layer,trainable=True)
print weights

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b, W, b]


In [12]:
network_output = lasagne.layers.get_output(output_layer)
#если вы используете дропаут - не забудьте продублировать всё в режиме deterministic=True

In [13]:
loss = lasagne.objectives.categorical_crossentropy(network_output,target_values).mean()

updates = lasagne.updates.adam(loss, weights)

# Компилируем всякое-разное

In [14]:
#обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence],network_output,allow_input_downcast=True)

# Генерируем свои законы

* Для этого последовательно применяем нейронку к своему же выводу.

* Генерировать можно по разному -
 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью
 * случайно, пропорционально softmax(probas*alpha), где alpha - "жадность"

In [15]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    
    return np.random.choice(np.arange(0,len(tokens)) ,p = probs)

In [16]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 

# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 

def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    Сгенерировать случайный текст при помощи сети

    sample_fun - функция, которая выбирает следующий сгенерированный токен
    
    seed_phrase - фраза, которую сеть должна продолжить. Если None - фраза выбирается случайно из corpora
    
    N - размер сгенерированного текста.
    
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print "Using random seed:",seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
           
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

# Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.



In [18]:
print("Training ...")

#сколько всего эпох
n_epochs=2

# раз в сколько эпох печатать примеры 
batches_per_epoch = 1000

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size=200

for epoch in xrange(n_epochs):

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
    if epoch%10==0:  
        print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))
        print "Генерируем текст в пропорциональном режиме"
        generate_sample(proportional_sample_fun,None)
        print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
        generate_sample(max_sample_fun,None)

Training ...
Epoch 0 average loss = 1.32543445492
Генерируем текст в пропорциональном режиме
Using random seed: прав 
----
 прав вынестра;тавина и сотрудиставоверенных законнае сроков складделянаны в пунтвошленных возмещения;
 срокромствованное средставу

 Налоговором помещения участвования кассация, юредитораейсти с договор 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: овари
----
 оварищественное поставляется с правонарушения обществляться по договором и иное применяться по правоможники определения правонарушения обязанности должностном сведения общего Кодекса или по применять в при 
----


# Конституция нового мирового правительства

In [18]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
 олжен согасшестве и на внутренных органами или создании ведется обеспечения хотя, пункта 1 статьей 293 настоящего Кодекса.

 Статья 767).
 В военности

 1. Стррения.

 1. В и вправе:
 в) пределенную граниченной определяется подели об административного имых для гакеслиатенных залового собакционерч 
----


In [19]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
 вения судебным устав и в уненными (– 2 составляет гражданина изъятии пролждным, этих до двухсот минимальных размерах, возместители;
 3) слова, общих гаранту или нарушения.

 Правой 29 настоящего Кодекса;
 3) наплажитая

 Статья 1004 №46-ФЗ) 
 2) разрешается сонстутского суда пользу находится лица, 
----


In [20]:
И далее по списку

SyntaxError: invalid syntax (<ipython-input-20-cb3e4dd1fd4b>, line 1)